# ESC 2020 SUMMER

## 데이터 분석 1차 발표 - Preprocessing & EDA
## 7조 
![sh](ChilliShrimp.png)

# Overview

[github link](https://github.com/njj06135/Baseball_ChilliShrimp)

- 조원 소개
- 분석 방식
- 향후 계획

![kbo](open_graph_kbo.png)

# 1. 조원 소개

- 조장 : ★곽현지★ ESC 22기



- 조원 : 오태환(22기) 김채형(23기) 박상재(24기) 이승준(24기)

# 2. 분석 방식

## 두 가지 제안!

1. 2020년 7월 19일 까지가 기준이니, 모든 년도의 팀 기록을 7월 19일 이전과 이후로 나누어서 7월 19일 이전은 X변수로 7월 19일 이후는 승률만 계산해 y로 넣고 2016~2019년의 데이터를 트레인 시킨 후 2020년 기록을 넣고 prediction 하자


|year|T_ID|PA|AB|H|...|
|---|---|---|---|---|---|
|2016(~7월 19일)|2592|320|280|100|...|
|2016(~7월 19일)|3244|350|300|120|...|
|...|...|...|...|...|...|
|2019(~7월 19일)|2342|340|310|100|...|

이걸 X값으로 하고

|Win(7월 19일 이후)|
|---|
|0.48|
|0.63|
|...|

이걸 Y값으로!

2. 팀 성적은 결국 선수 성적의 합이다. 선수들의 잔여 경기 성적을 예측한 뒤, 그것을 합쳐서 팀 성적을 만든 후 계산을 거쳐 승률을 계산하자. 타자와 투수의 성적을 합해 성적을 계산하기 때문에 자동으로 타율, 방어율은 계산이 된다.


1번 방식의 문제점은 row가 겨우 30개 밖에 안된다는 것이다! 이렇게 적은 데이터로는 좋은 결과를 낼 수 없다고 생각해서 2번이 좀 더 적절한 방식이라고 생각했다.

# 그렇다면 어떤 Y값을 추정해야 할까?

## 투수

투수의 경우는 명확한 y값이 있다. 바로 실점이다. 자책점이 아닌 실점을 사용한 이유는 결국 우리의 목적은 팀의 승률을 계산하는 것이기 때문이다. (참고 [자책점의 종말](https://www.beyondtheboxscore.com/2013/11/13/5080514/the-end-of-era-ra9-sabermetrics))
실책으로 인한 실점을 뺀 자책점을 사용하지 않고 실점을 사용한다면 팀의 수비에 대한 지표도 간접적으로 알 수 있을 것이다.

![R_ER](R_ER.png)

그래프에서 볼 수 있듯, 실점(R)쪽이 자책점(ER)보다 승률과의 상관관계가 더 높다. 예측 목표 중 하나인 평균 자책점은, 투수의 실점에 팀 실점-자책점 비율을 곱해 측정하도록 하자.

## 타자
그렇다면 타자의 y는 무엇으로 해야할까? 득점이나 타점은 선수 본인만의 능력이 아닌 팀 내 다른 선수들의 영향이 포함되어있다. 득점은 다른 선수가 쳐줘야 들어올 수 있고 타점은 다른 선수가 루상에 있어줘야 올릴 수 있기 때문이다. 그래서 그 선수가 얼마나 팀의 득점에 기여했는가를 어떻게 알 수 있을까를 찾아보던 중, XR이란 스탯을 발견하게 되었다.

# XR(Extrapolated Runs, 추정 득점)?

(1루타×0.5)+(2루타×0.72)+(3루타×1.04)+(홈런×1.44)+{(사구+볼넷－고의사구)×0.34}+(고의

   사구×0.25)+(도루×0.18)－(도루실패×0.32)－{(타수－안타－삼진)×0.09}－(삼진×0.098)－(병살타×0.37)+(희생

   플라이×0.37)+(희생번트×0.04)

출처: https://jagaevi.tistory.com/69 [기억상실증] 

이러한 복잡한 식으로 계산되는 XR은 Jim Furtado가 제안한 통계로, 그 선수가 팀의 득점에 얼마나 기여했는지를 회귀분석을 통해 계산한 값이다. 이와 비슷한 스탯으로는 세이버 메트릭스(sabermetrics, 야구 통계학이라고 생각하면 된다)의 대부 Bill James가 제안한 RC(Run Created)가 있는데, 직관으로 만든 RC보다 XR이 더 정확한 예측값을 제공한다고 한다.

https://www.baseballthinkfactory.org/btf/scholars/furtado/articles/IntroducingXR.htm

링크에서 볼 수 있듯, 팀 내 모든 선수의 XR을 더하면 팀의 득점과 거의 비슷한 값을 얻을 수 있다. 따라서 타자의 y로는 XR을 계산해 사용하도록 하겠다.

# X값들은 비율스탯으로 바꾸기

팀마다 경기수가 다르고 선수마다 들어간 타석수, 투입된 이닝 수가 다르다. 그렇기 때문에 절대적인 양으로 X값을 넣는다면 그러한 차이 때문에 왜곡된 값이 나올 수 있다고 생각했다. 그래서 타율, 출루율, 장타율, OPS, 9이닝당 볼넷, 9이닝당 삼진, 9이닝당 홈런 등 비율스탯으로 바꾸어 넣기로 했다. 대신 타석과 이닝은 포함시켜 절대적인 양도 반영하겠다. 

### OPS란?

OPS는 출루율과 장타율을 더한 값이다. 굉장히 직관적이고 쉽게 얻을 수 있는 지표일 뿐만 아니라, 팀 득점과의 상관관계가 상당히 높다. 
특히 KBO의 경우에 더 정교하다고 알려진 지표인 wOBA보다 득점과의 상관관계까 더 높다고 알려져있다.

![OPS](OPS_R.png)

출처 : [R로 보는 OPS와 득점의 상관관계](https://kuduz.tistory.com/1200)


# 2020년 7월 19일 이후의 X값은 어떻게 예측해야 할까?

## BABIP이론

[BABIP란 무엇인가](https://terms.naver.com/entry.nhn?docId=3572533&cid=58905&categoryId=58916)

![kim](kim.gif)

과연 이런 안타를 타자가 잘 친 것이라고 볼 수 있을까?

![hanhwa2](hanhwa2.gif)

또는 이런 안타를 투수가 잘 못던져서 맞은 것이라고 볼 수 있을까?

# 아니다! 

저런 것은 타자나 투수가 컨트롤하지 못하는 부분이다. 즉 **운** 이다.

시즌 별로 특출나게 운이 좋은 시즌이 있을 수 있고, 나쁜 시즌이 있을 수 있다. 하지만 통산 성적으로 누적해서 본다면, 그건 그 선수의 실력이지 않을까?

그런 생각에서 나온 주장이 'BABIP'이론이다. 다음 표는 타구별로 얼마나 타자, 투수, 수비가 기여하는지를 측정한 것이다.

![babip](babip.png)

즉 타자, 투수, 수비가 통제할 수 없는 순전히 '운에 의한' 결과가 11~13% 정도 나온다.(통계학에서 Reducible Error와 Irreducible Error라고 생각하면 좋을 것 같다) 

그래서 BABIP이론은 '인플레이 타율'로 그러한 운을 측정하려고 한다. 그라운드에 들어오지 못한 타구인 삼진, 홈런을 제외한 타구들 중 안타가 얼마나 나왔는지를 측정한 것이다. 

# 예측 지표로서의 BABIP

![babip2](babip2.jpg)

위의 표에서 볼 수 있듯, 통산 BABIP보다 높은 BABIP를 기록하면서 좋은 성적을 낸 선수는 그 다음 시즌 성적이 떨어지는 경향이 있다. 즉 운에 의해 성적이 좋아졌을 뿐, 운이 보통 수준으로만 돌아와도 성적이 떨어지게 되는 것이다. 

![bibip3](babip3.jpg)

투수의 경우도 마찬가지다. 본인의 평균 BABIP보다 낮은 BABIP를 기록하면 더 좋은 ERA을 보이고, 높은 경우는 안좋은 ERA를 보인다. 

출처 : https://terms.naver.com/entry.nhn?docId=3572533&cid=58905&categoryId=58916

# 그렇다면 BABIP를 어떻게 반영해볼까?

우리가 주어진 데이터는 2016~2020년 까지의 데이터이다. 이 기간 동안 선수별 BABIP를 구하도록 하겠다.
그리고 2020년 시즌에도 이 평균 BABIP를 기록한다고 가정하겠다. 즉 2016년 부터 2020년의 BABIP가 .350인데,  2020년 시즌에 7월 19일 이전 까지는 BABIP가 .380 정도의 높은 수준을 기록하고 있다면 BABIP를 .350에 회귀한다고 가정해 남은 기간 동안 더 안좋은 성적을 기록할 것이라고 예측하는 것이다.

예를 들면 타자의 경우는 이런 식으로 나타낼 수 있을 것 같다.

total_babip(2016~2020의 통산 BABIP) = (Inplay_hit_until_0719 + x) / (Inplay_until_0719 + Predicted_inplay)

이렇게 시계열을 통해 그 선수의 잔여 타석을 예측하고, 그 선수의 평균 인플레이타구 비율을 곱해 Predicted_inplay를 만든다. 

그리고 0719 이후의 인플레이 타구 안타 수를 미지수로 놓고, 2020시즌 BABIP가 통산 BABIP를 기록하도록 방정식을 풀어 안타의 갯수를 예측한다.

볼넷, 삼진, 2루타, 홈런 등은 그 선수의 평균 비율만큼을 이에 곱해 예측하도록 하자.

투수의 경우도 마찬가지이다. BABIP 이론에 따라 피안타를 예측한 다음, 비율에 따라 탈삼진, 볼넷, 피홈런 등을 예측하도록 하겠다.

# 그렇다면 팀 승률은 어떻게?

* [피타고리안 승률](https://sports.v.daum.net/v/20170802145321697)

* [한국프로야구에서의 피타고라스 정리의 정확도 측정, 이장택, 한국데이터정보과학회지 제26권 제3호](http://www.dbpia.co.kr.access.yonsei.ac.kr:8080/journal/articleDetail?nodeId=NODE07245164&language=ko_KR)

![pita](pita.png)

위의 과정을 통해 우리는 타자들의 XR들의 합으로 나타내어진 그 팀의 총 득점, 투수들의 실점의 합으로 나타내어진 그 팀의 총 실점을 예측했다. 그리고 팀의 득점과 실점을 통해 팀의 승률을 구하는 식이 바로 피타고리안 승률이다. 계산 식은 위와 같다.

밑에 링크를 걸어둔 논문을 보면, 2005~2014년 프로야구에서 피타고리안 승률은 실제 승률과 대략적으로 평균 1.95%밖에 차이가 나지 않는다!

또한 시즌의 70% 이상이 진행되었을 때 이러한 계산식이 유의하다고 한다. 그렇기 때문에 7월 19일 이후의 득실로 승률을 계산하기 보단, 7월 19일 이전의 실제 기록과 그 이후의 예측된 기록을 합쳐 시즌 승률을 구한 후, 역산을 통해 잔여 시즌의 승률을 구하도록 하겠다.

# 3. 향후 계획

- 추가 EDA : 시각화, Scaling, 파생변수 만들기
- 모델링
- 최종 데이터 만들기
- 혹시 입상을 하면 발표 ppt 만들기